In [1]:
using PyCall
using PyPlot

In [2]:
function read_lammps_output_file(lammps_output_filename::String)
    # read in lines of QE output file
    lammps_file = open(lammps_output_filename)
    lines = readlines(lammps_file)
    close(lammps_file)
    return lines
end

read_lammps_output_file (generic function with 1 method)

In [3]:
function get_thermo_data(lines::Array{String, 1})
    data_start = 0
    data_stop  = 0
    for (i, line) in enumerate(lines)
        if occursin("Step", line)
            data_start = i
        elseif occursin("Loop", line)
            data_stop  = i
        end
    end
    
    println("data_start = ", data_start)
    println("data_stop  = ", data_stop)
    
    num_data = data_stop - data_start - 1
    data = Dict{String, Array{Any, 1}}("Step"       => zeros(Int64, num_data),
                                       "v_eqtemp"   => zeros(Int64, num_data),
                                       "c_msd"      => zeros(Float64, num_data),
                                       "v_fitslope" => zeros(Float64, num_data)
                                      )
    
    for (i, line) in enumerate(lines[data_start+1:data_stop-1])
        data["Step"][i]       = parse(Int64, split(line)[1])
        data["v_eqtemp"][i]   = parse(Int64, split(line)[2])
        data["c_msd"][i]      = parse(Float64, split(line)[3])
        data["v_fitslope"][i] = parse(Float64, split(line)[4])
    end
    return data
end

get_thermo_data (generic function with 1 method)

In [4]:
# filename = joinpath(pwd(), "log.NiPyC2_experiment_with_Xe")

# lines = read_lammps_output_file(filename)

# sim_data = get_thermo_data(lines)

# length(sim_data["c_msd"])

In [5]:
# which xtals are we looking at
xtals = ["NiPyC2_experiment", "Pn_Ni-PyC-NH2"]
# which adsorbated are we condidering
adsorbates = ["Ar", "Kr", "Xe"]
# initialize Dict to store data
# Tuple is (xtal, gas)
sim_data = Dict{Tuple{String, String}, Dict{String, Array{Any, 1}}}()

# loop over xtals
for xtal in xtals
    # loop over adsorbates
    for gas in adsorbates
        # define filemane
        filename = joinpath(pwd(), "log." * xtal * "_with_" * gas)
        println(filename)
        # read file
        lines = read_lammps_output_file(filename)
        # get desired data and store in a Dict
        sim_data[(xtal, gas)] = get_thermo_data(lines)
        
        ###
        # Simulation output is in Å²/fs and we want to convert that to cm²/s
        # 1 Å  = 10⁻⁸cm
        # 1 fs = 10⁻¹⁵ s
        # 1 Å²/fs * (10¹⁵ fs/s)*(10⁻¹⁶ cm²/Å²) = 0.1 cm²/s
        # So, 1 cm²/s = 10 Å²/fs
        ###
        conversion_factor = 10 
        # apply conversion 
        sim_data[(xtal, gas)]["diffusion coeff. [cm²/s]"] = sim_data[(xtal, gas)]["v_fitslope"] * conversion_factor
    end
end

/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.NiPyC2_experiment_with_Ar
data_start = 135
data_stop  = 1137
/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.NiPyC2_experiment_with_Kr
data_start = 135
data_stop  = 1137
/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.NiPyC2_experiment_with_Xe
data_start = 143
data_stop  = 1145
/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.Pn_Ni-PyC-NH2_with_Ar
data_start = 149
data_stop  = 1151
/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.Pn_Ni-PyC-NH2_with_Kr
data_start = 149
data_stop  = 1151
/home/ng/NiPyC_analogues/lammps_diffusion_calcs/log.Pn_Ni-PyC-NH2_with_Xe
data_start = 149
data_stop  = 1151


In [15]:
# look at "NiPyC2_experiment" with Xe for example
last(sim_data[xtals[1], adsorbates[3]]["diffusion coeff. [cm²/s]"], 5)

5-element Vector{Any}:
  5.5193038e-12
  9.084335599999999e-13
 -2.5238458e-12
  2.6250733e-12
  2.3038841e-12

In [17]:
# look at "NiPyC2_experiment" with Kr for example
last(sim_data[xtals[1], adsorbates[2]]["diffusion coeff. [cm²/s]"], 5)

5-element Vector{Any}:
 1.1758968e-10
 1.2990202000000001e-10
 1.2756776e-10
 1.1880642e-10
 1.1590332e-10

In [ ]:
# Diffusive selectivity of Xe/Kr for 
# NiPyC2_experiment
S_parent_xekr = sim_data[xtals[1], adsorbates[3]]["diffusion coeff. [cm²/s]"][end] / 
                sim_data[xtals[1], adsorbates[2]]["diffusion coeff. [cm²/s]"][end]

In [12]:
# Pn_Ni-PyC-NH2
S_nh2_xekr = sim_data[xtals[2], adsorbates[3]]["diffusion coeff. [cm²/s]"][end] / 
             sim_data[xtals[2], adsorbates[2]]["diffusion coeff. [cm²/s]"][end]

0.019877636809713474